In [1]:
import collections
import os
import re
import openai
import pandas as pd
import pdfplumber
import torch
from fuzzywuzzy import fuzz
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import BertModel, BertTokenizer

dataDir = "../data/"
dataName = "Deep Learning.pdf"

from tqdm.notebook import tqdm

# openai.api_base = "https://api.chatanywhere.com.cn/"
openai.api_base = "https://api.chatanywhere.tech"
openai.api_key = "sk-LzwgVgu5xvNPpwoqCdeeVcAt7Tu7ZoZICXzzkheldIbXA60h"

e:\GitHub\kg\.conda\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# 1. 获取模型

In [2]:
tokenizer = BertTokenizer.from_pretrained(
    "bert-large-cased", cache_dir="../../../BERT/large"
)
model = BertModel.from_pretrained("bert-large-cased", cache_dir="../../../BERT/large")

# 2. 获取数据

In [3]:
with pdfplumber.open(dataDir + dataName) as f:
    # 目录架构生成
    c, p, n = [], [], []
    for i in range(7):
        page = f.pages[i]
        text = page.extract_text()
        text_split = text.split("\n")
        for i in text_split:
            if bool(re.match("[0-9]+\.[0-9]+", i.split(" ")[0])):
                c.append(i.split(" ")[0])
                p.append(int(i.split(" ")[-1]) + 15)
            if bool(re.match("[0-9]+", i.split(" ")[0])):
                for j in i.split(" "):
                    if bool(re.match("[A-Za-z]+", j)):
                        n.append((i.split(" ")[0], j))

p_range = list(zip(p, p[1:]))
p_range.append((735, 800))
c_p_range = list(zip(c, p_range))
index_dict = collections.defaultdict(list)
for k, v in c_p_range:
    index_dict[k.split(".")[0]].append((k, v))

with pdfplumber.open(dataDir + dataName) as f:
    content_dict = collections.defaultdict(list)

    for k, v in tqdm(index_dict.items(), total=len(index_dict)):
        for i in v:
            page_range = i[-1]
            if page_range[0] == page_range[1]:
                page_range = (page_range[0], page_range[1] + 1)
            for j in range(int(page_range[0]) - 1, int(page_range[1]) - 1):
                page = f.pages[j]

                text = page.extract_text().replace("\n", " ")

                content_dict[i[0]].append(text)

  0%|          | 0/20 [00:00<?, ?it/s]

# 3. 定义ChatGPT

In [4]:
class Chat:
    def __init__(self, conversation_list=[]):
        self.conversation_list = conversation_list
        self.costs_list = []

    def show_conversation(self, msg_list):
        for msg in msg_list[-2:]:
            if msg["role"] == "user":
                pass
            else:
                message = msg["content"]
                pass
                # print(f"\U0001f47D: {message}\n")

    def ask(self, prompt):
        self.conversation_list.append({"role": "user", "content": prompt})
        openai.api_key = "sk-LzwgVgu5xvNPpwoqCdeeVcAt7Tu7ZoZICXzzkheldIbXA60h"
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-16k", messages=self.conversation_list
        )
        answer = response.choices[0].message["content"]

        self.conversation_list.append({"role": "assistant", "content": answer})
        self.show_conversation(self.conversation_list)

        # cost = total_counts(response)
        # self.costs_list.append(cost)
        return answer

# 4. 设计Prompt

In [11]:
NER_prompt = f"""
角色：
你是一个深度学习领域的实体标注专员

任务：
给定字符串，请找出全部深度学习领域的实体

步骤：
请以以下步骤执行：
1. 找出句子中的所有深度学习领域的实体
2. 依次检查实体是否属于深度学习领域
3. 将属于深度学习领域的实体返回
4. 若没有深度学习领域的实体，则返回()

格式：
请以以下格式返回：
(entity1, entity2, ...)

举例如下：
An illustration of how the gradient descent algorithm uses the derivatives of a function can be used to follow the function downhill to a minimum.
(gradient descent algorithm)

an encoder or reader or input RNN processes the input sequence. The encoder emits the context C, usually as a simple function of its final hidden state.
(encoder, RNN, hidden state)

There is no constraint that the encoder must have the same size of hidden layer as the decoder
(hidden layer, decoder)

Computer vision has traditionally been one of the most active research areas for deep learning applications, because vision is a task that is effortless for humans and many animals but challenging for computers (Ballard et al., 1983)
(Computer vision, deep learning)

Dataset augmentation may be seen as a way of preprocessing the training set only.
(Dataset augmentation)

CHAPTER 1. INTRODUCTION of the flowchart of the computations needed to compute the representation of each concept may be much deeper than the graph of the concepts themselves.
()

注意事项：
1. 请严格遵循字符串中的原本表述
2. 除返回结果外，不要返回任何其他内容
"""

In [12]:
check_prompt = f"""
任务：
请检查所给实体是否属于深度学习领域

格式：
请以以下格式返回：
如果该实体是深度学习领域的实体，返回True，否则返回False

举例如下：
input: deep learning
output: True

input: AI system
output: True

input: image
output: False

input: Image Net
output: True

input: face
output: False

注意事项：
1. 请仅返回True或者False，不要返回任何其他内容
"""

# 5. 处理数据，获取sample

In [13]:
def find_position(content_token, entity_token):
    """
    Finds the position of an entity token within a content token.

    Args:
        content_token (torch.Tensor): A tensor representing the content token.
        entity_token (torch.Tensor): A tensor representing the entity token.

    Returns:
        torch.Tensor: A tensor representing the position of the entity token within the content token.
                      Each element in the tensor represents the position of a token in the content token:
                      - 0: Token does not match the entity token.
                      - 1: Token matches the entity token, but is not the first token.
                      - 2: Token matches the entity token and is the first token.
    """
    position = torch.zeros_like(content_token)
    for entity in entity_token:
        for i in range(len(content_token) - len(entity) + 1):
            if torch.all(content_token[i : i + len(entity)] == entity):
                position[i] = 2
                position[i + 1 : i + len(entity)] = 1
    return position

In [18]:
def process_content(content_dict, content_list):
    """
    Process the content dictionary to extract named entities and save them to a CSV file.

    Args:
        content_dict (dict): A dictionary containing the content to process.

    Returns:
        None
    """
    total = 0
    for k, v in content_dict.items():
        if k in content_list:
            print(k+':', end='\n')
            if total >= 1500:
                break
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=100, chunk_overlap=20
            )
            docs = text_splitter.split_text(" ".join(i for i in v))

            # Process each document
            for index, content in enumerate(docs):
                total += 1
                if total == 1500:
                    break

                # Initialize NER chatbot
                if index % 5 == 0:
                    conversation_list = [
                        {
                            "role": "system",
                            "content": NER_prompt,
                        }
                    ]
                    bot_ner = Chat(conversation_list)

                # Extract named entities using NER chatbot
                answer_ner = bot_ner.ask("input: " + content)
                entity_list_temp = re.sub("\(|\)|", "", answer_ner).split(", ")
                # Initialize check chatbot
                conversation_list = [
                    {
                        "role": "system",
                        "content": check_prompt,
                    }
                ]
                bot_check = Chat(conversation_list)

                entity_list = []
                # Check if each entity is valid using check chatbot
                for e in entity_list_temp:
                    answer_check = bot_check.ask("input: " + e)
                    if answer_check == "True":
                        entity_list.append(e)
                        
                if entity_list:
                    # Tokenize content and entities
                    content_token = tokenizer(content, return_tensors="pt")[
                        "input_ids"
                    ].squeeze(0)
                    entity_token = []
                    for e in entity_list:
                        entity_token.append(
                            tokenizer(e, return_tensors="pt")["input_ids"].squeeze(0)[1:-1]
                        )

                    # Find position of entities in content
                    label = find_position(content_token, entity_token)

                    # Save the results to a CSV file
                    df = pd.DataFrame(
                        [
                            [
                                tokenizer.batch_decode(content_token),
                                tokenizer.batch_decode(entity_token),
                                label,
                            ]
                        ],
                        columns=["text", "entity", "label"],
                    )
                    df.to_csv(
                        os.path.join(dataDir + "/relations", f"sample.csv"),
                        mode="a",
                        header=not os.path.exists(
                            os.path.join(dataDir + "/relations", f"sample.csv")
                        ),
                        index=False,
                    )
                    # Print the content, tokens, entity list, and label
                    print(
                        "content: " + str(content),
                        "content_token: " + str(content_token),
                        "entity_list: " + str(entity_list),
                        "entity_token: " + str(entity_token),
                        sep="\n",
                    )
                    print("label: " + str(label))
                    print(
                        "----------------------------------------------------------------------------------------------------------"
                    )

In [19]:
content_list = ["7.1", "8.1", "9.1", "10.1", "11.1", "12.1"]
process_content(content_dict, content_list)

6.1:
content: CHAPTER 6. DEEP FEEDFORWARD NETWORKS mappings from x to y that lack feedback connections. Other
content_token: tensor([  101,  8203,   127,   119, 18581, 16668,   143, 27073, 16395,  9565,
        11840, 23354, 26546,  1942,  2924,  9565, 25370, 13970,  1116,  1121,
          193,  1106,   194,  1115,  2960, 13032,  6984,   119,  2189,   102])
entity_list: ['deep feedforward networks']
entity_token: [tensor([ 1996,  4877, 14467,  1197,  5984,  6379])]
label: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0])
----------------------------------------------------------------------------------------------------------
content: simple example of a feedforward network. Next, we address each of the design decisions needed to
content_token: tensor([  101,  3014,  1859,  1104,   170,  4877, 14467,  1197,  5984,  2443,
          119,  5893,   117,  1195,  4134,  1296,  1104,  1103,  1902,  6134,
         1834,  1106,   102])
en

APIError: The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_e3353435f5d2cd1db667be34c402d97c in your email.) {"error":{"message":"The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_e3353435f5d2cd1db667be34c402d97c in your email.)","type":"server_error","param":null,"code":null}} 500 {'error': {'message': 'The server had an error processing your request. Sorry about that! You can retry your request, or contact us through our help center at help.openai.com if you keep seeing this error. (Please include the request ID req_e3353435f5d2cd1db667be34c402d97c in your email.)', 'type': 'server_error', 'param': None, 'code': None}} {'Server': 'Tengine', 'Content-Type

In [161]:
df = pd.read_csv(dataDir + "relations/" + "sample.csv", index_col=False)